In [5]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [6]:
df = sns.load_dataset('penguins')
dfcopia = df.copy()
dfcopia = dfcopia.dropna(axis=0)
especies = dfcopia['species'].unique()
print(especies)

dfcopia['specie numerica'] = 0

i = 0
for pinguin in dfcopia['species']:
    #print(pinguin)
    if pinguin == 'Adelie':
        dfcopia['specie numerica'].iloc[i] = 1
        i += 1
    elif pinguin == 'Chinstrap':
        dfcopia['specie numerica'].iloc[i] = 2
        i += 1
    elif pinguin == 'Gentoo':
        dfcopia['specie numerica'].iloc[i] = 3
        i += 1
    #print(dfcopia['species'].iloc[i-1],dfcopia['specie numerica'].iloc[i-1])

features_numericos = ["bill_length_mm","bill_depth_mm", "flipper_length_mm", "body_mass_g"]
df_numericos = dfcopia[features_numericos]

['Adelie' 'Chinstrap' 'Gentoo']


In [9]:
sc = StandardScaler()
sc.fit(df_numericos)
X_norm = sc.transform(df_numericos)

In [10]:
print(dfcopia)

    species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen            39.1           18.7              181.0   
1    Adelie  Torgersen            39.5           17.4              186.0   
2    Adelie  Torgersen            40.3           18.0              195.0   
4    Adelie  Torgersen            36.7           19.3              193.0   
5    Adelie  Torgersen            39.3           20.6              190.0   
..      ...        ...             ...            ...                ...   
338  Gentoo     Biscoe            47.2           13.7              214.0   
340  Gentoo     Biscoe            46.8           14.3              215.0   
341  Gentoo     Biscoe            50.4           15.7              222.0   
342  Gentoo     Biscoe            45.2           14.8              212.0   
343  Gentoo     Biscoe            49.9           16.1              213.0   

     body_mass_g     sex  specie numerica  
0         3750.0    Male                1  

In [11]:
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]
TARGET = ["specie numerica"]

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() #Definindo o normalizador
X_norm = scaler.fit_transform(dfcopia[FEATURES].values) #Normalizando

Normalizamos os dados a fim de realizarmos os PCA.

In [30]:
df_norm = pd.DataFrame(X_norm,columns = FEATURES) #Criando o DataFrame a partir com dados normalizados, que é um array de numpy
df_norm = pd.concat([df_norm,pd.Series(dfcopia['specie numerica']).reset_index(drop=True)],axis=1) #Concatenando a coluna 'price' ao DataFrame
print(df_norm.head(10)) #Exibindo as 10 primeiras linhas do dataset normalizado

   bill_length_mm  bill_depth_mm  flipper_length_mm  specie numerica
0        0.254545       0.666667           0.152542                1
1        0.269091       0.511905           0.237288                1
2        0.298182       0.583333           0.389831                1
3        0.167273       0.738095           0.355932                1
4        0.261818       0.892857           0.305085                1
5        0.247273       0.559524           0.152542                1
6        0.258182       0.773810           0.389831                1
7        0.327273       0.535714           0.169492                1
8        0.236364       0.964286           0.322034                1
9        0.090909       0.952381           0.440678                1


In [14]:
from sklearn.decomposition import PCA
import numpy as np

X = df_norm[FEATURES] #Definindo o X como as features
y = df_norm['specie numerica']  #Definindo o y como o target

pca = PCA(n_components=3) #Calculando o PCA para três componentes
pca.fit(X)                #Ajustando o PCA
X_pca = pca.transform(X)  #Realizando o PCA

In [15]:
print(np.sum(pca.explained_variance_ratio_)) #Calculando a variância explicada acumulada para as três componentes principais

0.9999999999999999


É notável que a variância é muito alta, então devemos dividir o dataset. 

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( #Chamando a função train_test_split, atribuindo os valores nas variáveis a esquerda da igualdade
    X_pca,                                           #Dados das componentes principais do PCA
    y,                                               #Dados do target
    test_size=0.2,                                   #Tamanho do conjunto de teste
    random_state=61)                                 #Semente aleatória

In [23]:
df_norm_treino = pd.DataFrame(X_pca,columns=['PC1', 'PC2', 'PC3']) #Define as colunas das PC em um dataframe, utilizando os dados do X_pca
df_norm_treino['specie numerica'] = y                                        #Adiciona a coluna preço, com os dados de y
print(df_norm_treino.head(10))                                     #Exibe as 10 primeiras linhas do dataset
corrmat = df_norm_treino.corr()                                    #Cálculo da correlação
print(round(corrmat,4))                                          #Exibe a correlação com 4 casas decimais

        PC1       PC2       PC3  specie numerica
0 -0.417304 -0.047414  0.056515                1
1 -0.263773 -0.135362  0.062671                1
2 -0.185816 -0.033273 -0.050103                1
3 -0.355020 -0.010007 -0.164409                1
4 -0.437199  0.154948 -0.125524                1
5 -0.359367 -0.131234  0.090425                1
6 -0.312010  0.081679 -0.143602                1
7 -0.298952 -0.093937  0.137469                1
8 -0.477330  0.194728 -0.178856                1
9 -0.451868  0.116134 -0.347823                1
                    PC1     PC2     PC3  specie numerica
PC1              1.0000 -0.0000  0.0000           0.9474
PC2             -0.0000  1.0000  0.0000           0.0304
PC3              0.0000  0.0000  1.0000           0.1284
specie numerica  0.9474  0.0304  0.1284           1.0000


Com isso, criamos um novo dataset com dimensionalidade reduzida.

In [31]:
from sklearn.linear_model import LinearRegression

# cria o modelo
modelo_linear = LinearRegression()

# treina o modelo
modelo_linear.fit(X_train, y_train)

LinearRegression()

In [25]:
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_test
y_previsao = modelo_linear.predict(X_test)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False) #Cálculo do RMSE

print(f"O RMSE do modelo linear foi de {RMSE}.")

O RMSE do modelo linear foi de 0.27231019633925413.


In [26]:
from sklearn.ensemble import RandomForestRegressor

# cria o modelo
SEMENTE_ALEATORIA = 61454
modelo_rf = RandomForestRegressor(random_state=SEMENTE_ALEATORIA)

# treina o modelo
modelo_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=61454)

In [27]:
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_test
y_previsao = modelo_rf.predict(X_test)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo árvore de decisão foi de {RMSE}.")

O RMSE do modelo árvore de decisão foi de 0.19409952824690646.


O RMSE obtido significa que o erro é de 19%, aproximadamente.